In [1]:
# os.environ["MODIN_ENGINE"] = "dask"
# import modin.pandas as pd
import pandas as pd
import numpy as np
import requests
import math

In [2]:
from credentials import IEX_CLOUD_API_TOKEN

In [3]:
stocks = pd.read_csv("sp-500-stocks.csv")

In [4]:
columns = ["Tickers", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns=columns)

In [ ]:
final_dataframe = pd.DataFrame(columns=columns)
for stock in stocks["Ticker"][:20]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [stock, data["latestPrice"], data["marketCap"], "N/A"], 
            index=columns
        ),
        ignore_index=True
    )
final_dataframe

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
    # print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns=columns)
for symbol_string in symbol_strings:
    batch_api_call_url = \
        f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]["quote"]["latestPrice"], 
                    data[symbol]["quote"]["marketCap"],
                    "N/A"
                ], 
                index=columns
            ), 
            ignore_index=True
        )
final_dataframe

In [ ]:
portfolio_size = input("Enter the value of your portfolio:")
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number!\nPlease try again ...")
    portfolio_size = input("Enter the value of your portfolio:")
    val = float(portfolio_size)

In [ ]:
position_size = val / len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])
final_dataframe


In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

In [ ]:
string_format = writer.book.add_format(
    {
        # 'font_color': '#000000',
        # 'bg_color': '#ffffff',
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        # 'font_color': '#000000',
        # 'bg_color': '#ffffff',
        'border': 1
    }
)
int_format = writer.book.add_format(
    {
        'num_format': '0',
        # 'font_color': '#000000',
        # 'bg_color': '#ffffff',
        'border': 1
    }
)
columns_formats = {
    'A': ['Tickers', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shared to Buy', int_format],
}

for column in columns_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, columns_formats[column][1])

In [ ]:
writer.save()